#Task 1

Since the dataset contains many heterogeneous data, this task is about preparing (pre-processing) all the
above tables containing **ONLY** active patients. For each step report the number of patients remaining
and the class distribution. This task requires you to complete the following action items:


1. *Select events of interest* - we want only patients with at least one cardiovascular event in their trajectories.
2. *Invalid feature cleaning* - check for dates and time intervals $[x,y]$ such that $y < x$ and not $x \leq y$.
Check for years that do not make sense (e.g., events before the birth of a particular patient).
3. *Remove patients with all dates in the same month* - we only want patients that have a long trajectory of examinations and diagnoses.
4. *Modify the actual ranges of* ***esamilaboratorioparameteri*** - see Table 2.
5. *Cohort selection and label definition* - use only those patients that after all the previous steps contain
at least two events before calculating the label. Let $\mathcal{P} = \{p_1, \dots p_n\}$ be the set of all patients in the dataset.
Let $d(e^i_k)$ be the date of the last event $e_k$ for patient $p_i \in \mathcal{P}$. The label of the patient
$p_i$ is calculated as follows:

\\begin{equation}
	y(p_i) = \begin{cases}
		1 \ \text{  if, within } d(e^i_k) - 6 \text{ months, }\
    p_i \text{ has a cardiovascular event }\\
		0 \text{ otherwise}
	\end{cases}
\end{equation}
>Eliminate the patients that have a trajectory shorter than or equal to 6 months.

6. **Concentration** - consider other cleaning strategies that improve the dataset's quality. How do
you measure the quality before and after performing your cleaning strategy?

<br>
<br>
<center>
  <table>
<caption>Table 2: The true ranges of the AMD/STITCH codes.</caption>
 <tr><th> Code </th><th> Descriptive name </th> <th> True range </th></tr>
<tr><td>AMD004</td><td> Systolic blood pressure</td><td> $40$ $\leq$ $x$ $\leq$ $200$</td></tr>
<tr><td>AMD005</td><td> Diastolic blood pressure</td><td> $40$ $\leq$ $x$ $\leq$ $130$</td></tr>
<tr><td>AMD007</td><td> Fasting blood glucose</td><td> $50$ $\leq$ $x$ $\leq$ $500$</td></tr>
<tr><td>AMD008</td><td> HbA1c</td><td> $5$ $\leq$ $x$ $\leq$ $15$</td></tr>
<tr><td>AMD009</td><td> Creatininemia</td><td> Not available
<tr><td>AMD111</td><td> Microalbuminuria</td><td> Not available</td></tr>
<tr><td>STITCH001</td><td> BMI</td><td> Not available
<tr><td>STITCH002</td><td> LDL Cholesterol</td><td> $30$ $\leq$ $x$ $\leq$ $300$</td></tr>
<tr><td>STITCH003</td><td> Non-HDL Cholesterlo</td><td> $60$ $\leq$ $x$ $\leq$ $330$</td></tr>
<tr><td>STITCH004</td><td> eGFR MDRD</td><td> Not available</td></tr>
<tr><td>STITCH005</td><td> eGFR CKD-EPI</td><td> Not available</td></tr>
</table>
</center>

<br>


In [ ]:
from google.colab import drive

# mount the drive
drive.mount('/content/drive', force_remount=True)

path="/content/drive/MyDrive/project/(1999250)_EHR"


Mounted at /content/drive


In [ ]:
# Import required packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [ ]:
path = path + '/data/sample'

In [ ]:
active_patients_info = pd.read_csv(path + '/anagraficapazientiattivi.csv')
diagnosis_tests = pd.read_csv(path + '/diagnosi.csv')
laboratory_tests_calculated_parameters = pd.read_csv(path + '/esamilaboratorioparametricalcolati.csv')
laboratory_tests = pd.read_csv(path + '/esamilaboratorioparametri.csv')
medical_tests = pd.read_csv(path + '/esamistrumentali.csv')
prescriptions_of_diabetes_drugs = pd.read_csv(path + '/prescrizionidiabetefarmaci.csv')
prescriptions_of_non_diabetes_drugs = pd.read_csv(path + '/prescrizionidiabetenonfarmaci.csv')
patients_diets_and_blood_glucose_controls = pd.read_csv(path + '/prescrizioninondiabete.csv')
# amd_meaning = pd.read_csv('raw_sampled_dataset/data/amd_codes_for_bert.csv')
# atc_info = pd.read_csv('raw_sampled_dataset/data/atc_info_nodup.csv')
dataset=[active_patients_info,
         diagnosis_tests,
         laboratory_tests_calculated_parameters,
         laboratory_tests,
         medical_tests,
         prescriptions_of_diabetes_drugs,
         prescriptions_of_non_diabetes_drugs,
         patients_diets_and_blood_glucose_controls]

In [ ]:
def dataset_shapes():
  '''
  This is done to visualize more easily the change in dimensionality after filtering
  '''
  print("active_patients_info: ", active_patients_info.shape)
  print("diagnosis_tests: ", diagnosis_tests.shape)
  print("laboratory_tests: ", laboratory_tests.shape)
  print("laboratory_tests_calculated_parameters: ", laboratory_tests_calculated_parameters.shape)
  print("medical_tests: ", medical_tests.shape)
  print("prescriptions_of_diabetes_drugs: ", prescriptions_of_diabetes_drugs.shape)
  print("prescriptions_of_non_diabetes_drugs: ", prescriptions_of_non_diabetes_drugs.shape)
  print("patients_diets_and_blood_glucose_controls: ", patients_diets_and_blood_glucose_controls.shape)

In [ ]:
def clean_dataframe(dataframe):
  '''
  function used to drop duplicates
  '''
  dataframe.drop('Unnamed: 0', axis=1, inplace=True)
  dataframe.drop_duplicates(inplace=True)
  dataframe.reset_index(drop=True, inplace=True)

In [ ]:
print("Before:")
dataset_shapes()

clean_dataframe(active_patients_info)
clean_dataframe(diagnosis_tests)
clean_dataframe(laboratory_tests)
clean_dataframe(laboratory_tests_calculated_parameters)
clean_dataframe(medical_tests)
clean_dataframe(prescriptions_of_diabetes_drugs)
clean_dataframe(prescriptions_of_non_diabetes_drugs)
clean_dataframe(patients_diets_and_blood_glucose_controls)

print("After:")
dataset_shapes()

Before:
active_patients_info:  (250000, 13)
diagnosis_tests:  (4427337, 6)
laboratory_tests:  (28628530, 6)
laboratory_tests_calculated_parameters:  (10621827, 7)
medical_tests:  (1015740, 6)
prescriptions_of_diabetes_drugs:  (7012648, 8)
prescriptions_of_non_diabetes_drugs:  (548467, 6)
patients_diets_and_blood_glucose_controls:  (5083861, 6)
After:
active_patients_info:  (250000, 12)
diagnosis_tests:  (4427337, 5)
laboratory_tests:  (28628502, 5)
laboratory_tests_calculated_parameters:  (8698045, 6)
medical_tests:  (1015740, 5)
prescriptions_of_diabetes_drugs:  (7012648, 7)
prescriptions_of_non_diabetes_drugs:  (548467, 5)
patients_diets_and_blood_glucose_controls:  (5083861, 5)


In [ ]:
# Renaming column names
active_patients_info.columns = ['id_center', 'id_ana', 'sex', 'diagnosis_of_diabetes_year', 'type_of_diabetes', 'education', 'marital_status', 'profession', 'origin', 'birth_year', 'first_access_year', 'death_year']
prescriptions_of_non_diabetes_drugs.columns = ['id_center', 'id_ana', 'date', 'amd_code', 'value']
patients_diets_and_blood_glucose_controls.columns = ['id_center', 'id_ana', 'date', 'amd_code', 'value']
prescriptions_of_diabetes_drugs.columns = ['id_center', 'id_ana', 'date', 'atc_code', 'quantity', 'meal_id', 'drug_prescription']
medical_tests.columns = ['id_center', 'id_ana', 'date', 'amd_code', 'value']
laboratory_tests_calculated_parameters.columns = ['id_center', 'id_ana', 'date', 'amd_code', 'value', 'stitch_code']
laboratory_tests.columns = ['id_center', 'id_ana', 'date', 'amd_code', 'value']
diagnosis_tests.columns = ['id_center', 'id_ana', 'date', 'amd_code', 'value']

## 1. Select events of interest

Macro-cardiovascular events:
* AMD047: Myocardial infarction
* AMD048: Coronary angioplasty
* AMD049: Coronary bypass
* AMD071: Ictus
* AMD081: Lower limb angioplasty
* AMD082: Peripheral By-pass Lower Limbs
* AMD208: Revascularization of intracranial and neck vessels
* AMD303: Ischemic stroke


In [ ]:
macro_events = ['AMD047','AMD048','AMD049','AMD071','AMD081','AMD082','AMD208','AMD303']
cv_diagnoses = diagnosis_tests[diagnosis_tests.amd_code.isin(macro_events)]
interesting_patients = cv_diagnoses[['id_center','id_ana']].drop_duplicates().reset_index(drop=True)
interesting_patients.head()

,id_center,id_ana
0,1,5
1,1,36
2,1,38
3,1,61
4,1,65


In [ ]:
interesting_patients.shape

(50000, 2)

In [ ]:
def filter_dataframe(target, interesting_patients):
  return target.merge(interesting_patients, how='inner', on=['id_center','id_ana'])[target.columns]

In [ ]:
print("Before:")
dataset_shapes()

active_patients_info = filter_dataframe(active_patients_info, interesting_patients)
diagnosis_tests = filter_dataframe(diagnosis_tests, interesting_patients)
laboratory_tests_calculated_parameters = filter_dataframe(laboratory_tests_calculated_parameters, interesting_patients)
laboratory_tests = filter_dataframe(laboratory_tests, interesting_patients)
medical_tests = filter_dataframe(medical_tests, interesting_patients)
prescriptions_of_diabetes_drugs = filter_dataframe(prescriptions_of_diabetes_drugs, interesting_patients)
prescriptions_of_non_diabetes_drugs = filter_dataframe(prescriptions_of_non_diabetes_drugs, interesting_patients)
patients_diets_and_blood_glucose_controls = filter_dataframe(patients_diets_and_blood_glucose_controls, interesting_patients)

print("After:")
dataset_shapes()

Before:
active_patients_info:  (250000, 12)
diagnosis_tests:  (4427337, 5)
laboratory_tests:  (28628502, 5)
laboratory_tests_calculated_parameters:  (8698045, 6)
medical_tests:  (1015740, 5)
prescriptions_of_diabetes_drugs:  (7012648, 7)
prescriptions_of_non_diabetes_drugs:  (548467, 5)
patients_diets_and_blood_glucose_controls:  (5083861, 5)
After:
active_patients_info:  (50000, 12)
diagnosis_tests:  (1938342, 5)
laboratory_tests:  (7371151, 5)
laboratory_tests_calculated_parameters:  (2279156, 6)
medical_tests:  (290793, 5)
prescriptions_of_diabetes_drugs:  (1989613, 7)
prescriptions_of_non_diabetes_drugs:  (150340, 5)
patients_diets_and_blood_glucose_controls:  (1995073, 5)


## 2. Invalid feature cleaning

In [ ]:
active_patients_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id_center                   50000 non-null  int64  
 1   id_ana                      50000 non-null  int64  
 2   sex                         50000 non-null  object 
 3   diagnosis_of_diabetes_year  49474 non-null  float64
 4   type_of_diabetes            50000 non-null  int64  
 5   education                   14784 non-null  float64
 6   marital_status              24860 non-null  float64
 7   profession                  22936 non-null  float64
 8   origin                      584 non-null    float64
 9   birth_year                  50000 non-null  int64  
 10  first_access_year           43026 non-null  float64
 11  death_year                  5800 non-null   float64
dtypes: float64(7), int64(4), object(1)
memory usage: 5.0+ MB


Great amounts of NaN values in some of these variables. This might invalidate the use of these variables as prediction features in future steps.



In [ ]:
# Test 1: birth_year <= death_year
null_death = active_patients_info.death_year.isnull()
birth_before_death = active_patients_info.birth_year <= active_patients_info.death_year
birth_death_mask = null_death | birth_before_death

# Tes 2: birth_year <= first_access_year <= death_year
null_death = active_patients_info.death_year.isnull()
null_first_access = active_patients_info.first_access_year.isnull()
first_access_after_birth = active_patients_info.birth_year <= active_patients_info.first_access_year
first_access_before_death = active_patients_info.first_access_year <= active_patients_info.death_year
first_access_mask = null_first_access | (first_access_after_birth & (null_death | first_access_before_death))

# Test 3: birth_year <= diagnosis_of_diabetes_year <= death_year
null_death = active_patients_info.death_year.isnull()
null_diagnosis_of_diabetes = active_patients_info.diagnosis_of_diabetes_year.isnull()
diagnosis_of_diabetes_after_birth = active_patients_info.birth_year <= active_patients_info.diagnosis_of_diabetes_year
diagnosis_of_diabetes_before_death = active_patients_info.diagnosis_of_diabetes_year <= active_patients_info.death_year
diagnosis_of_diabetes_mask = null_diagnosis_of_diabetes | (diagnosis_of_diabetes_after_birth & (null_death | diagnosis_of_diabetes_before_death))

print("Before: ", active_patients_info.shape)
active_patients_info = active_patients_info.drop(
    active_patients_info[~(birth_death_mask & first_access_mask & diagnosis_of_diabetes_mask)].index).reset_index(drop=True)
print("After: ", active_patients_info.shape)


Before:  (50000, 12)
After:  (49990, 12)


Now we need to filter the remaining tables and remove the patients that didn't fulfill the invalid features tests

In [ ]:
print("Before:")
dataset_shapes()

diagnosis_tests = filter_dataframe(diagnosis_tests, active_patients_info)
laboratory_tests_calculated_parameters = filter_dataframe(laboratory_tests_calculated_parameters, active_patients_info)
laboratory_tests = filter_dataframe(laboratory_tests, active_patients_info)
medical_tests = filter_dataframe(medical_tests, active_patients_info)
prescriptions_of_diabetes_drugs = filter_dataframe(prescriptions_of_diabetes_drugs, active_patients_info)
prescriptions_of_non_diabetes_drugs = filter_dataframe(prescriptions_of_non_diabetes_drugs, active_patients_info)
patients_diets_and_blood_glucose_controls = filter_dataframe(patients_diets_and_blood_glucose_controls, active_patients_info)

print("After:")
dataset_shapes()

Before:
active_patients_info:  (49990, 12)
diagnosis_tests:  (1938342, 5)
laboratory_tests:  (7371151, 5)
laboratory_tests_calculated_parameters:  (2279156, 6)
medical_tests:  (290793, 5)
prescriptions_of_diabetes_drugs:  (1989613, 7)
prescriptions_of_non_diabetes_drugs:  (150340, 5)
patients_diets_and_blood_glucose_controls:  (1995073, 5)
After:
active_patients_info:  (49990, 12)
diagnosis_tests:  (1938123, 5)
laboratory_tests:  (7370649, 5)
laboratory_tests_calculated_parameters:  (2278985, 6)
medical_tests:  (290769, 5)
prescriptions_of_diabetes_drugs:  (1989453, 7)
prescriptions_of_non_diabetes_drugs:  (150323, 5)
patients_diets_and_blood_glucose_controls:  (1994962, 5)


In [ ]:
# Forcing the correct datatype for dates.
diagnosis_tests.date = pd.to_datetime(diagnosis_tests.date)
laboratory_tests_calculated_parameters.date = pd.to_datetime(laboratory_tests_calculated_parameters.date)
laboratory_tests.date = pd.to_datetime(laboratory_tests.date)
medical_tests.date = pd.to_datetime(medical_tests.date)
prescriptions_of_diabetes_drugs.date = pd.to_datetime(prescriptions_of_diabetes_drugs.date)
prescriptions_of_non_diabetes_drugs.date = pd.to_datetime(prescriptions_of_non_diabetes_drugs.date)
patients_diets_and_blood_glucose_controls.date = pd.to_datetime(patients_diets_and_blood_glucose_controls.date)

In [ ]:
# birth_year <= event_date <= death_year
def between_birth_and_death(events_dataframe, active_patients_info, name):
    '''
    Function that ensures that no event happened before the birth year of the patient
    or after its death year.
    '''
    initial_size = events_dataframe.shape[0]
    patients_birth_death = active_patients_info[['id_center', 'id_ana','birth_year', 'death_year']]
    events_dataframe = pd.merge(events_dataframe, patients_birth_death, on=['id_center', 'id_ana'], how='inner')

    null_death = events_dataframe.death_year.isnull()
    event_after_birth = events_dataframe.birth_year <= events_dataframe['date'].dt.year
    event_between = events_dataframe['date'].dt.year.between(
        events_dataframe.birth_year, events_dataframe.death_year)
    result = events_dataframe[(~null_death & event_between) | (null_death & event_after_birth)]
    print("Removed " + str(initial_size - result.shape[0]) + " rows out of " + str(initial_size) + " from " + name)

    return result

In [ ]:
prescriptions_of_non_diabetes_drugs = between_birth_and_death(
    prescriptions_of_non_diabetes_drugs, active_patients_info,
    'prescriptions_of_non_diabetes_drugs').reset_index(drop=True)
patients_diets_and_blood_glucose_controls = between_birth_and_death(
    patients_diets_and_blood_glucose_controls,active_patients_info,
    'patients_diets_and_blood_glucose_controls').reset_index(drop=True)
prescriptions_of_diabetes_drugs = between_birth_and_death(
    prescriptions_of_diabetes_drugs, active_patients_info,
    'prescriptions_of_diabetes_drugs').reset_index(drop=True)
medical_tests = between_birth_and_death(
    medical_tests, active_patients_info,
    'medical_tests').reset_index(drop=True)
laboratory_tests_calculated_parameters = between_birth_and_death(
    laboratory_tests_calculated_parameters, active_patients_info,
    'laboratory_tests_calculated_parameters').reset_index(drop=True)
laboratory_tests = between_birth_and_death(
    laboratory_tests, active_patients_info,
    'laboratory_tests').reset_index(drop=True)
diagnosis_tests = between_birth_and_death(
    diagnosis_tests, active_patients_info,
    'diagnosis_tests').reset_index(drop=True)

Removed 36 rows out of 150323 from prescriptions_of_non_diabetes_drugs
Removed 271 rows out of 1994962 from patients_diets_and_blood_glucose_controls
Removed 295 rows out of 1989453 from prescriptions_of_diabetes_drugs
Removed 37 rows out of 290769 from medical_tests
Removed 342 rows out of 2278985 from laboratory_tests_calculated_parameters
Removed 1045 rows out of 7370649 from laboratory_tests
Removed 13458 rows out of 1938123 from diagnosis_tests


## 3. Remove patients with all dates in the same month

In [ ]:
diagnosis_tests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1924665 entries, 0 to 1924664
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id_center   int64         
 1   id_ana      int64         
 2   date        datetime64[ns]
 3   amd_code    object        
 4   value       object        
 5   birth_year  int64         
 6   death_year  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 102.8+ MB


In [ ]:
def decompose_date(dataframe):
  '''
  Function used to decompose the dates into day, month and year
  '''
  dataframe = dataframe.copy()
  dataframe['day'] = dataframe['date'].dt.day
  dataframe['month'] = dataframe['date'].dt.month
  dataframe['year'] = dataframe['date'].dt.year
  return dataframe

In [ ]:
diagnosis_tests = decompose_date(diagnosis_tests)
laboratory_tests = decompose_date(laboratory_tests)
laboratory_tests_calculated_parameters = decompose_date(laboratory_tests_calculated_parameters)
medical_tests = decompose_date(medical_tests)

In [ ]:
'''
We create a dataframe that contains all the events of all the patients through all the tables
'''

trajectories = pd.concat(
    [diagnosis_tests[["id_center", "id_ana", "month", "year"]],
     laboratory_tests[["id_center", "id_ana", "month", "year"]],
     laboratory_tests_calculated_parameters[["id_center", "id_ana", "month", "year"]],
     medical_tests[["id_center", "id_ana", "month", "year"]]])

In [ ]:
trajectories

,id_center,id_ana,month,year
0,1,5,1,1980
1,1,5,1,1986
2,1,5,1,1987
3,1,5,1,1987
4,1,5,12,1997
...,...,...,...,...
290727,500,8692111,1,2012
290728,500,8692111,10,2012
290729,500,8692111,1,2014
290730,500,8692111,8,2014


In [ ]:
# We group the observations by id_ana and id_center (both form the patients' id)
# and count for each patient the number of different months and years that every instance has
groups = trajectories.groupby(['id_ana', 'id_center']).nunique()
groups

month  year
id_ana  id_center             
1       77             6     7
        121           12    15
        143            9     8
2       5              9     8
        8             12    14
...                  ...   ...
770252  500            2     2
874692  500            2     1
1255182 500            6     6
1282277 500            7     9
8692111 500            5     6

[49989 rows x 2 columns]

In [ ]:
trajectories = trajectories.set_index(['id_ana', 'id_center'])
trajectories

month  year
id_ana  id_center             
5       1              1  1980
        1              1  1986
        1              1  1987
        1              1  1987
        1             12  1997
...                  ...   ...
8692111 500            1  2012
        500           10  2012
        500            1  2014
        500            8  2014
        500            6  2015

[11863644 rows x 2 columns]

If all the examination and diagnosis' events of a patient are in the same month of the same year, then we can remove that the patient. This happens when the patient only has simultaneously one unique month and one unique year as dates of its event. It has to be simultaneous because the patient can have all the events in july and therefore will have just one month but if those events happened in different years the conditions won't hold and the patient won't be removed.






In [ ]:
patients_to_be_removed = groups[(groups.month==1) & (groups.year==1)]
patients_to_be_removed

month  year
id_ana id_center             
50     112            1     1
57     6              1     1
       173            1     1
66     60             1     1
119    213            1     1
...                 ...   ...
33020  111            1     1
34352  111            1     1
34409  111            1     1
34783  111            1     1
36248  111            1     1

[897 rows x 2 columns]

In [ ]:
to_remove = pd.merge(interesting_patients, patients_to_be_removed, on=['id_center', 'id_ana'], how='inner')[['id_center', 'id_ana']]
to_remove

,id_center,id_ana
0,4,5958
1,4,6112
2,5,1702
3,5,5483
4,6,57
...,...,...
892,151,1469
893,176,435
894,176,3975
895,234,8345


In [ ]:
interesting_patients.shape

(50000, 2)

In [ ]:
# Removing patients from interesting_patients that didn't fulfill the previous requirement
merged = interesting_patients.merge(to_remove, how='left', indicator=True)
interesting_patients = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)
interesting_patients.shape

(49103, 2)

In [ ]:
print("Before:")
dataset_shapes()

active_patients_info = filter_dataframe(active_patients_info, interesting_patients)
diagnosis_tests = filter_dataframe(diagnosis_tests, interesting_patients)
laboratory_tests_calculated_parameters = filter_dataframe(laboratory_tests_calculated_parameters, interesting_patients)
laboratory_tests = filter_dataframe(laboratory_tests, interesting_patients)
medical_tests = filter_dataframe(medical_tests, interesting_patients)
prescriptions_of_diabetes_drugs = filter_dataframe(prescriptions_of_diabetes_drugs, interesting_patients)
prescriptions_of_non_diabetes_drugs = filter_dataframe(prescriptions_of_non_diabetes_drugs, interesting_patients)
patients_diets_and_blood_glucose_controls = filter_dataframe(patients_diets_and_blood_glucose_controls, interesting_patients)

print("After:")
dataset_shapes()

Before:
active_patients_info:  (49990, 12)
diagnosis_tests:  (1924665, 10)
laboratory_tests:  (7369604, 10)
laboratory_tests_calculated_parameters:  (2278643, 11)
medical_tests:  (290732, 10)
prescriptions_of_diabetes_drugs:  (1989158, 9)
prescriptions_of_non_diabetes_drugs:  (150287, 7)
patients_diets_and_blood_glucose_controls:  (1994691, 7)
After:
active_patients_info:  (49093, 12)
diagnosis_tests:  (1919045, 10)
laboratory_tests:  (7361304, 10)
laboratory_tests_calculated_parameters:  (2276064, 11)
medical_tests:  (290314, 10)
prescriptions_of_diabetes_drugs:  (1986333, 9)
prescriptions_of_non_diabetes_drugs:  (150031, 7)
patients_diets_and_blood_glucose_controls:  (1991552, 7)


## 4. Modify the actual ranges of esamilaboratorioparameteri

This is done by clipping the values that are below the lower limit of the range, to that lower limit. Conversely, values that are above the upper limit of the range are clipped to the upper limit.

In [ ]:
true_ranges = pd.DataFrame(data={
    'code': ['AMD004', 'AMD005', 'AMD007', 'AMD008',
             'STITCH002', 'STITCH003'],
    'lb': [40, 40, 50, 5, 30, 60],
    'ub': [200, 130, 500, 15, 300, 330]
})

In [ ]:
# the print is just to be sure that the shape didn't change
print("Before: ", laboratory_tests.shape[0])
df = pd.merge(laboratory_tests, true_ranges, left_on='amd_code', right_on='code', how='inner')
df = df.drop('code', axis=1)
laboratory_tests.value.update(
    df.value.clip(lower=df.lb, upper=df.ub))
print("After: ", laboratory_tests.shape[0])

Before:  7361304
After:  7361304


## 5. Cohort selection and label definition

In [ ]:
trajectories = pd.concat(
    [diagnosis_tests[["id_center", "id_ana", "date"]],
     laboratory_tests[["id_center", "id_ana", "date"]],
     laboratory_tests_calculated_parameters[["id_center", "id_ana", "date"]],
     medical_tests[["id_center", "id_ana", "date"]],
     prescriptions_of_non_diabetes_drugs[["id_center", "id_ana", "date"]],
     patients_diets_and_blood_glucose_controls[["id_center", "id_ana", "date"]],
     prescriptions_of_diabetes_drugs[["id_center", "id_ana", "date"]]])

trajectories

,id_center,id_ana,date
0,1,5,1980-01-01
1,1,5,1986-01-01
2,1,5,1987-01-01
3,1,5,1987-01-01
4,1,5,1997-12-01
...,...,...,...
1986328,82,749,2014-10-24
1986329,82,749,2015-04-03
1986330,82,749,2015-04-03
1986331,82,749,2015-04-03


In [ ]:
'''
We group the elements by patient_id  and count for each group the number of instances (rows)
Those patients who had only 1 event, are discarded.
'''

groups = trajectories.groupby(['id_ana', 'id_center']).size().reset_index(name='count')
groups

,id_ana,id_center,count
0,1,77,88
1,1,121,1944
2,1,143,442
3,2,5,346
4,2,8,1134
...,...,...,...
49087,770252,500,9
49088,874692,500,25
49089,1255182,500,141
49090,1282277,500,64


In [ ]:
# We select the patients that had at least 2 events in their trajectories
interesting_patients = groups[groups["count"] > 1]
interesting_patients

,id_ana,id_center,count
0,1,77,88
1,1,121,1944
2,1,143,442
3,2,5,346
4,2,8,1134
...,...,...,...
49087,770252,500,9
49088,874692,500,25
49089,1255182,500,141
49090,1282277,500,64


In [ ]:
trajectories  = filter_dataframe(trajectories, interesting_patients)

In [ ]:
trajectories = trajectories.set_index(['id_ana', 'id_center'])
trajectories

date
id_ana id_center           
5      1         1980-01-01
       1         1986-01-01
       1         1987-01-01
       1         1987-01-01
       1         1997-12-01
...                     ...
5259   233       2018-01-30
       233       2018-06-06
       233       2018-11-27
       233       2019-05-24
       233       2018-01-30

[15974643 rows x 1 columns]

Eliminate the patients that have a trajectory shorter than or equal to 6 months.

In [ ]:
'''
We identify the earliest and latest date of an event within the trajectory of each patient.
And create these new columns with the just found dates
'''
trajectories["begin"] = trajectories.groupby(['id_ana', 'id_center']).date.min()
trajectories["end"] = trajectories.groupby(['id_ana', 'id_center']).date.max()

In [ ]:
trajectories

date      begin        end
id_ana id_center                                 
5      1         1980-01-01 1980-01-01 2013-12-31
       1         1986-01-01 1980-01-01 2013-12-31
       1         1987-01-01 1980-01-01 2013-12-31
       1         1987-01-01 1980-01-01 2013-12-31
       1         1997-12-01 1980-01-01 2013-12-31
...                     ...        ...        ...
5259   233       2018-01-30 2018-01-30 2019-05-24
       233       2018-06-06 2018-01-30 2019-05-24
       233       2018-11-27 2018-01-30 2019-05-24
       233       2019-05-24 2018-01-30 2019-05-24
       233       2018-01-30 2018-01-30 2019-05-24

[15974643 rows x 3 columns]

In [ ]:
'''
Here we remove the patients who have a trajectory shorter than 6 months. The logic is simple
just keep those patients where the last event happened 6 months after its the first recorded event
'''
trajectories = trajectories[(trajectories['begin'] + pd.DateOffset(months=6)) < trajectories['end']]
trajectories.sort_index()

date      begin        end
id_ana  id_center                                 
1       77        2001-10-29 2001-10-29 2007-10-12
        77        2001-10-29 2001-10-29 2007-10-12
        77        2002-03-15 2001-10-29 2007-10-12
        77        2002-03-15 2001-10-29 2007-10-12
        77        2002-03-18 2001-10-29 2007-10-12
...                      ...        ...        ...
8692111 500       2012-10-03 1990-10-03 2015-06-29
        500       2012-10-03 1990-10-03 2015-06-29
        500       2012-10-03 1990-10-03 2015-06-29
        500       2012-10-03 1990-10-03 2015-06-29
        500       2012-10-03 1990-10-03 2015-06-29

[15918821 rows x 3 columns]

In [ ]:
trajectories.shape

(15918821, 3)

In [ ]:
trajectories = trajectories.drop(columns=['begin', 'date'])
trajectories = trajectories.reset_index()
trajectories = trajectories.drop_duplicates()
trajectories

,id_ana,id_center,end
0,5,1,2013-12-31
1039,36,1,2010-06-25
1287,38,1,2015-12-18
1811,61,1,2019-05-15
2762,65,1,2017-11-09
...,...,...,...
15918394,9136,253,2014-10-01
15918595,13892,253,2019-04-18
15918706,1026,267,2006-09-19
15918759,8862,71,2006-10-18


In [ ]:
'''
We filter the trajectories that have a cardiovascular events
'''
labels_dataframe = diagnosis_tests.merge(trajectories, on=['id_center', 'id_ana'], how='inner')[["id_center", "id_ana", "date", "amd_code", "end"]]
labels_dataframe

,id_center,id_ana,date,amd_code,end
0,1,5,1980-01-01,AMD247,2013-12-31
1,1,5,1986-01-01,AMD247,2013-12-31
2,1,5,1987-01-01,AMD083,2013-12-31
3,1,5,1987-01-01,AMD247,2013-12-31
4,1,5,1997-12-01,AMD247,2013-12-31
...,...,...,...,...,...
1909659,250,10961,2014-01-26,AMD071,2018-07-10
1909660,251,12146,2017-09-30,AMD071,2019-03-21
1909661,253,9136,2007-12-12,AMD071,2014-10-01
1909662,253,13892,2015-01-09,AMD071,2019-04-18


In [ ]:
has_cv_event = labels_dataframe["amd_code"].isin(macro_events) # has a cardiovascular event
event_within_6_months = labels_dataframe["date"].between(
    labels_dataframe["end"] - pd.DateOffset(months=6),
    labels_dataframe["end"])

In [ ]:
'''
Patients are labelled with a 1 if and only if they fulfill, simultaneously, two conditions:
1. they must have had a cardiovascular event
2. the event must have happened within the last 6 months of its recorded history

Otherwise, the patient is labelled with a 0.
'''
labels_dataframe["label"] = np.where(has_cv_event & event_within_6_months, 1, 0)
labels_dataframe

,id_center,id_ana,date,amd_code,end,label
0,1,5,1980-01-01,AMD247,2013-12-31,0
1,1,5,1986-01-01,AMD247,2013-12-31,0
2,1,5,1987-01-01,AMD083,2013-12-31,0
3,1,5,1987-01-01,AMD247,2013-12-31,0
4,1,5,1997-12-01,AMD247,2013-12-31,0
...,...,...,...,...,...,...
1909659,250,10961,2014-01-26,AMD071,2018-07-10,0
1909660,251,12146,2017-09-30,AMD071,2019-03-21,0
1909661,253,9136,2007-12-12,AMD071,2014-10-01,0
1909662,253,13892,2015-01-09,AMD071,2019-04-18,0


In [ ]:
labels_dataframe = (labels_dataframe.groupby(["id_center", "id_ana"]).label.sum() > 0).reset_index()

In [ ]:
labels_dataframe

,id_center,id_ana,label
0,1,5,True
1,1,36,False
2,1,38,False
3,1,61,False
4,1,65,True
...,...,...,...
47792,500,610835,False
47793,500,672954,False
47794,500,1255182,False
47795,500,1282277,False


In [ ]:
'Finally labelling the patients!'

labels_dataframe["label"] = np.where(labels_dataframe.label==True, 1, 0)
labels_dataframe

,id_center,id_ana,label
0,1,5,1
1,1,36,0
2,1,38,0
3,1,61,0
4,1,65,1
...,...,...,...
47792,500,610835,0
47793,500,672954,0
47794,500,1255182,0
47795,500,1282277,0


In [ ]:
labels_dataframe["label"].value_counts()

0    32370
1    15427
Name: label, dtype: int64

In [ ]:
trajectories = pd.concat(
    [diagnosis_tests[["id_center", "id_ana", "date"]],
     laboratory_tests[["id_center", "id_ana", "date"]],
     laboratory_tests_calculated_parameters[["id_center", "id_ana", "date"]],
     medical_tests[["id_center", "id_ana", "date"]],
     prescriptions_of_non_diabetes_drugs[["id_center", "id_ana", "date"]],
     patients_diets_and_blood_glucose_controls[["id_center", "id_ana", "date"]],
     prescriptions_of_diabetes_drugs[["id_center", "id_ana", "date"]]])

trajectories = trajectories.merge(labels_dataframe, on=['id_center', 'id_ana'], how='inner')


In [ ]:
trajectories

,id_center,id_ana,date,label
0,1,5,1980-01-01,1
1,1,5,1986-01-01,1
2,1,5,1987-01-01,1
3,1,5,1987-01-01,1
4,1,5,1997-12-01,1
...,...,...,...,...
15918754,267,1026,2006-08-02,0
15918755,267,1026,2006-08-02,0
15918756,267,1026,2006-09-19,0
15918757,267,1026,2006-09-19,0


In [ ]:
trajectories["label"].value_counts()

0    10837195
1     5081564
Name: label, dtype: int64

In [ ]:
print("Before:")
dataset_shapes()

active_patients_info = filter_dataframe(active_patients_info, labels_dataframe)
diagnosis_tests = filter_dataframe(diagnosis_tests, labels_dataframe)
laboratory_tests_calculated_parameters = filter_dataframe(laboratory_tests_calculated_parameters, labels_dataframe)
laboratory_tests = filter_dataframe(laboratory_tests, labels_dataframe)
medical_tests = filter_dataframe(medical_tests, labels_dataframe)
prescriptions_of_diabetes_drugs = filter_dataframe(prescriptions_of_diabetes_drugs, labels_dataframe)
prescriptions_of_non_diabetes_drugs = filter_dataframe(prescriptions_of_non_diabetes_drugs, labels_dataframe)
patients_diets_and_blood_glucose_controls = filter_dataframe(patients_diets_and_blood_glucose_controls, labels_dataframe)

print("After:")
dataset_shapes()

Before:
active_patients_info:  (49093, 12)
diagnosis_tests:  (1919045, 10)
laboratory_tests:  (7361304, 10)
laboratory_tests_calculated_parameters:  (2276064, 11)
medical_tests:  (290314, 10)
prescriptions_of_diabetes_drugs:  (1986333, 9)
prescriptions_of_non_diabetes_drugs:  (150031, 7)
patients_diets_and_blood_glucose_controls:  (1991552, 7)
After:
active_patients_info:  (47797, 12)
diagnosis_tests:  (1909664, 10)
laboratory_tests:  (7337343, 10)
laboratory_tests_calculated_parameters:  (2268428, 11)
medical_tests:  (289176, 10)
prescriptions_of_diabetes_drugs:  (1980368, 9)
prescriptions_of_non_diabetes_drugs:  (149422, 7)
patients_diets_and_blood_glucose_controls:  (1984358, 7)


In [ ]:
# Dropping columns
diagnosis_tests = diagnosis_tests.drop(columns=['day', 'month', 'year'])
laboratory_tests_calculated_parameters = laboratory_tests_calculated_parameters.drop(columns=['day', 'month', 'year'])
laboratory_tests = laboratory_tests.drop(columns=['day', 'month', 'year'])
medical_tests = medical_tests.drop(columns=['day', 'month', 'year'])

In [ ]:
active_patients_info = active_patients_info.merge(labels_dataframe, on=['id_center', 'id_ana'], how='inner')

In [ ]:
# Writing the tables!
#diagnosis_tests.to_csv('data/preprocessed/diagnosis_tests.csv', mode='w', header=True, index=False)
#laboratory_tests_calculated_parameters.to_csv('data/preprocessed/laboratory_tests_calculated_parameters.csv', mode='w', header=True, index=False)
#laboratory_tests.to_csv('data/preprocessed/laboratory_tests.csv', mode='w', header=True, index=False)
#medical_tests.to_csv('data/preprocessed/medical_tests.csv', mode='w', header=True, index=False)
#active_patients_info.to_csv('data/preprocessed/active_patients_info.csv', mode='w', header=True, index=False)
#prescriptions_of_diabetes_drugs.to_csv('data/preprocessed/prescriptions_of_diabetes_drugs.csv', mode='w', header=True, index=False)
#prescriptions_of_non_diabetes_drugs.to_csv('data/preprocessed/prescriptions_of_non_diabetes_drugs.csv', mode='w', header=True, index=False)
#patients_diets_and_blood_glucose_controls.to_csv('data/preprocessed/patients_diets_and_blood_glucose_controls.csv', mode='w', header=True, index=False)
#trajectories.to_csv('data/preprocessed/trajectories.csv', mode='w', header=True, index=False)